# Data Processing

This notebook aims at processing the data from the CAISO

In [2]:
import pandas as pd
import os

os.chdir(r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project\data")


- CAISO 

In [11]:
CAISO_lmp=pd.read_csv(r"data/us Markets/CAISO/hourly_avg_lmp.csv")
CAISO_lmp

,Time,Interval Start,Interval End,Market,Location,Location Type,LMP,Energy,Congestion,Loss
0,2022-07-01 00:00:00-07:00,2022-07-01 00:00:00-07:00,2022-07-01 01:00:00-07:00,DAY_AHEAD_HOURLY,TH_NP15_GEN-APND,Trading Hub,73.70168,70.70564,2.95362,0.04242
1,2022-07-01 00:00:00-07:00,2022-07-01 00:00:00-07:00,2022-07-01 01:00:00-07:00,DAY_AHEAD_HOURLY,TH_SP15_GEN-APND,Trading Hub,61.42091,70.70564,-5.72823,-3.55649
2,2022-07-01 00:00:00-07:00,2022-07-01 00:00:00-07:00,2022-07-01 01:00:00-07:00,DAY_AHEAD_HOURLY,TH_ZP26_GEN-APND,Trading Hub,65.16609,70.70564,-2.23759,-3.30195
3,2022-07-01 01:00:00-07:00,2022-07-01 01:00:00-07:00,2022-07-01 02:00:00-07:00,DAY_AHEAD_HOURLY,TH_NP15_GEN-APND,Trading Hub,67.25023,65.39378,1.79759,0.05885
4,2022-07-01 01:00:00-07:00,2022-07-01 01:00:00-07:00,2022-07-01 02:00:00-07:00,DAY_AHEAD_HOURLY,TH_SP15_GEN-APND,Trading Hub,59.82000,65.39378,-2.55258,-3.02119
...,...,...,...,...,...,...,...,...,...,...
65443,2025-09-30 22:00:00-07:00,2025-09-30 22:00:00-07:00,2025-09-30 23:00:00-07:00,DAY_AHEAD_HOURLY,TH_SP15_GEN-APND,Trading Hub,38.24687,42.11953,-1.70771,-2.16494
65444,2025-09-30 22:00:00-07:00,2025-09-30 22:00:00-07:00,2025-09-30 23:00:00-07:00,DAY_AHEAD_HOURLY,TH_ZP26_GEN-APND,Trading Hub,40.48108,42.11953,0.00000,-1.63845
65445,2025-09-30 23:00:00-07:00,2025-09-30 23:00:00-07:00,2025-10-01 00:00:00-07:00,DAY_AHEAD_HOURLY,TH_NP15_GEN-APND,Trading Hub,40.28451,41.22205,-0.75204,-0.18550
65446,2025-09-30 23:00:00-07:00,2025-09-30 23:00:00-07:00,2025-10-01 00:00:00-07:00,DAY_AHEAD_HOURLY,TH_SP15_GEN-APND,Trading Hub,37.89718,41.22205,-1.51110,-1.81377


In [8]:
#CAISO LMP streamlining
CAISO_lmp.drop(columns=['Interval Start','Interval End','Market','Energy','Congestion','Loss'], inplace=True)
CAISO_lmp['Year'] = CAISO_lmp['Time'].str[:4].astype(int)
CAISO_lmp['Month'] = CAISO_lmp['Time'].str[5:7].astype(int)
CAISO_lmp['Day'] = CAISO_lmp['Time'].str[8:10].astype(int)
CAISO_lmp['Hour'] = CAISO_lmp['Time'].str[11:13].astype(int)
CAISO_lmp['Timezone'] = CAISO_lmp['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
CAISO_lmp = CAISO_lmp[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'Location', 'LMP']]

# Calculate average LMP across all locations for each time
CAISO_lmp_avg = CAISO_lmp.groupby(['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone']).agg({
    'LMP': 'mean'
}).reset_index()


print(CAISO_lmp_avg.head())

# Save the averaged data
CAISO_lmp_avg.to_csv(r"data/us Markets/CAISO/CAISO_lmp.csv", index=False)


                        Time  Year  Month  Day  Hour Timezone        LMP
0  2022-07-01 00:00:00-07:00  2022      7    1     0   -07:00  66.762893
1  2022-07-01 01:00:00-07:00  2022      7    1     1   -07:00  62.648637
2  2022-07-01 02:00:00-07:00  2022      7    1     2   -07:00  61.347333
3  2022-07-01 03:00:00-07:00  2022      7    1     3   -07:00  59.363743
4  2022-07-01 04:00:00-07:00  2022      7    1     4   -07:00  59.856077


In [15]:
#Now streamlining CAISO_REN_DAM_Forecasts

import zipfile
# Define the folder containing the .zip files
input_dir = r"CAISO_REN_DAM_Forecasts"


# Extract all .zip files into separate subfolders
def extract_zip_files(input_dir):
    for file in os.listdir(input_dir):
        if file.endswith(".zip"):
            zip_path = os.path.join(input_dir, file)
            extract_folder = os.path.join(input_dir, file.replace(".zip", ""))  # Create a subfolder for each .zip file
            os.makedirs(extract_folder, exist_ok=True)  # Ensure the subfolder exists
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)  # Extract into the subfolder
            print(f"✅ Extracted: {file} to {extract_folder}")

# Call the function to extract files
extract_zip_files(input_dir)

✅ Extracted: CAISO_DAM_Forecast_2022_07.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_07
✅ Extracted: CAISO_DAM_Forecast_2022_08.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_08
✅ Extracted: CAISO_DAM_Forecast_2022_09.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_09
✅ Extracted: CAISO_DAM_Forecast_2022_10.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_10
✅ Extracted: CAISO_DAM_Forecast_2022_11.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_11
✅ Extracted: CAISO_DAM_Forecast_2022_12.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_12
✅ Extracted: CAISO_DAM_Forecast_2023_01.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2023_01
✅ Extracted: CAISO_DAM_Forecast_2023_02.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2023_02
✅ Extracted: CAISO_DAM_Forecast_2023_03.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2023_03
✅ Extracted: CAISO_DAM_Forecast_2023_04.zip to CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2023_04
✅ Extracted: CAISO_D

In [32]:
# Define the path to the file
file_path = r"CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_07\20220701_20220731_SLD_REN_FCST_DAM_20251010_15_11_03_v1.csv"

# Read the file using pandas
forecast_data = pd.read_csv(file_path)



# Read the file using pandas
forecast_data = pd.read_csv(file_path)



# Assuming 'df' is your DataFrame
columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                   'LABEL', 'XML_DATA_ITEM', 'MARKET_RUN_ID_POS', 'RENEW_POS', 'MARKET_RUN_ID', 'GROUP']

# Drop the columns
df = forecast_data.drop(columns=columns_to_drop)

# Sort the DataFrame by OPR_DT and OPR_HR in ascending order
df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)

# Group by OPR_DT, OPR_HR, and RENEWABLE_TYPE, then take the sum
grouped_df = df.groupby(['OPR_DT', 'OPR_HR', 'RENEWABLE_TYPE'], as_index=False).sum()
grouped_df.drop(columns=['TRADING_HUB'], inplace=True)
# Extract Year, Month, and Day from OPR_DT
grouped_df['Year'] = grouped_df['OPR_DT'].str[:4].astype(int)
grouped_df['Month'] = grouped_df['OPR_DT'].str[5:7].astype(int)
grouped_df['Day'] = grouped_df['OPR_DT'].str[8:10].astype(int)

# Reorder columns to place Year, Month, and Day at the beginning
columns_order = ['Year', 'Month', 'Day'] + [col for col in grouped_df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
grouped_df = grouped_df[columns_order]
grouped_df = grouped_df.rename(columns={'OPR_HR':'Hour'})

# Display the updated DataFrame
print(grouped_df)

      Year  Month  Day  Hour RENEWABLE_TYPE       MW
0     2022      7    1     1          Solar     0.00
1     2022      7    1     1           Wind  3644.68
2     2022      7    1     2          Solar     0.00
3     2022      7    1     2           Wind  3568.87
4     2022      7    1     3          Solar     0.00
...    ...    ...  ...   ...            ...      ...
1435  2022      7   30    22           Wind  1874.36
1436  2022      7   30    23          Solar     0.00
1437  2022      7   30    23           Wind  1876.18
1438  2022      7   30    24          Solar     0.00
1439  2022      7   30    24           Wind  1817.47

[1440 rows x 6 columns]


In [35]:
import os
import pandas as pd

# Define the base folder containing all monthly subfolders
base_dir = r"CAISO_REN_DAM_Forecasts"

# Function to process a single CSV file
def process_csv(file_path):
    # Read the file
    forecast_data = pd.read_csv(file_path)
    
    # Drop unnecessary columns
    columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                       'LABEL', 'XML_DATA_ITEM', 'MARKET_RUN_ID_POS', 'RENEW_POS', 'MARKET_RUN_ID', 'GROUP']
    df = forecast_data.drop(columns=columns_to_drop)
    
    # Sort by OPR_DT and OPR_HR
    df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)
    
    # Group by OPR_DT, OPR_HR, and RENEWABLE_TYPE, then take the mean
    grouped_df = df.groupby(['OPR_DT', 'OPR_HR', 'RENEWABLE_TYPE'], as_index=False).sum()
    grouped_df.drop(columns=['TRADING_HUB'], inplace=True)
    
    # Extract Year, Month, and Day from OPR_DT
    grouped_df['Year'] = grouped_df['OPR_DT'].str[:4].astype(int)
    grouped_df['Month'] = grouped_df['OPR_DT'].str[5:7].astype(int)
    grouped_df['Day'] = grouped_df['OPR_DT'].str[8:10].astype(int)
    
    # Reorder columns to place Year, Month, and Day at the beginning
    columns_order = ['Year', 'Month', 'Day'] + [col for col in grouped_df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
    grouped_df = grouped_df[columns_order]

    
    return grouped_df

# Iterate through all subfolders and process each CSV file
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):  # Check if it's a folder
        # Extract the date (e.g., "2022_07") from the folder name
        folder_date = folder.split('_')[-2:]  # Extract the last two parts (e.g., "2022" and "07")
        folder_date_str = '_'.join(folder_date)  # Join them back as "2022_07"
        
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(folder_path, file)
                print(f"Processing file: {file_path}")
                
                # Process the CSV file
                processed_df = process_csv(file_path)
                
                # Save the processed DataFrame to a new CSV file
                output_file = os.path.join(folder_path, f"CAISO_DAM_REN_Forecast_{folder_date_str}_processed.csv")
                processed_df.to_csv(output_file, index=False)
                print(f"✅ Processed and saved: {output_file}")

Processing file: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_07\20220701_20220731_SLD_REN_FCST_DAM_20251010_15_11_03_v1.csv
✅ Processed and saved: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_07\CAISO_DAM_REN_Forecast_2022_07_processed.csv
Processing file: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_08\20220801_20220831_SLD_REN_FCST_DAM_20251010_15_11_13_v1.csv
✅ Processed and saved: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_08\CAISO_DAM_REN_Forecast_2022_08_processed.csv
Processing file: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_09\20220901_20220930_SLD_REN_FCST_DAM_20251010_15_11_15_v1.csv
✅ Processed and saved: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_09\CAISO_DAM_REN_Forecast_2022_09_processed.csv
Processing file: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_10\20221001_20221031_SLD_REN_FCST_DAM_20251010_15_11_21_v1.csv
✅ Processed and saved: CAISO_REN_DAM_Forecasts\CAISO_DAM_Forecast_2022_10\CAISO_DAM_REN_Forecast_2022_10_processed.csv
Processing f

In [36]:
#Now streamlining CAISO_DAM_Load_Forecasts

import zipfile
# Define the folder containing the .zip files
input_dir = r"CAISO_DAM_Load_Forecasts"


# Extract all .zip files into separate subfolders
def extract_zip_files(input_dir):
    for file in os.listdir(input_dir):
        if file.endswith(".zip"):
            zip_path = os.path.join(input_dir, file)
            extract_folder = os.path.join(input_dir, file.replace(".zip", ""))  # Create a subfolder for each .zip file
            os.makedirs(extract_folder, exist_ok=True)  # Ensure the subfolder exists
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)  # Extract into the subfolder
            print(f"✅ Extracted: {file} to {extract_folder}")

# Call the function to extract files
extract_zip_files(input_dir)

✅ Extracted: CAISO_DAM_Load_Forecast_2022_07.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_07
✅ Extracted: CAISO_DAM_Load_Forecast_2022_08.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_08
✅ Extracted: CAISO_DAM_Load_Forecast_2022_09.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_09
✅ Extracted: CAISO_DAM_Load_Forecast_2022_10.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_10
✅ Extracted: CAISO_DAM_Load_Forecast_2022_11.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_11
✅ Extracted: CAISO_DAM_Load_Forecast_2022_12.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_12
✅ Extracted: CAISO_DAM_Load_Forecast_2023_01.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2023_01
✅ Extracted: CAISO_DAM_Load_Forecast_2023_02.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2023_02
✅ Extracted: CAISO_DAM_Load_Forecast_2023_03.zip to CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2023_03
✅ Extracted: CAISO_

In [43]:
# Define the path to the file
file_path = r"CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_07\20220701_20220731_SLD_FCST_DAM_20251012_14_55_39_v1.csv"

# Read the file using pandas
forecast_data = pd.read_csv(file_path)



# Read the file using pandas
forecast_data = pd.read_csv(file_path)



# Assuming 'df' is your DataFrame
columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                   'LOAD_TYPE', 'MARKET_RUN_ID', 'LABEL','XML_DATA_ITEM','POS','EXECUTION_TYPE','GROUP']
# Drop the columns
df = forecast_data.drop(columns=columns_to_drop)

# Sort the DataFrame by OPR_DT and OPR_HR in ascending order
df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)
#For each day, keep only the CA ISO-TAC AREA_TYPE
df = df[df['TAC_AREA_NAME'] == 'CA ISO-TAC']
df=df.reset_index(drop=True)
df.drop(columns=['TAC_AREA_NAME'], inplace=True)
df['Year'] = df['OPR_DT'].str[:4].astype(int)
df['Month'] = df['OPR_DT'].str[5:7].astype(int)
df['Day'] = df['OPR_DT'].str[8:10].astype(int)

# Reorder columns to place Year, Month, and Day at the beginning
columns_order = ['Year', 'Month', 'Day'] + [col for col in df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
df = df[columns_order]
df = df.rename(columns={'OPR_HR':'Hour'})

# Display the updated DataFrame
print(df)


     Year  Month  Day  Hour        MW
0    2022      7    1     1  26808.84
1    2022      7    1     2  25299.89
2    2022      7    1     3  24244.10
3    2022      7    1     4  23523.73
4    2022      7    1     5  23440.70
..    ...    ...  ...   ...       ...
715  2022      7   30    20  34538.91
716  2022      7   30    21  34012.17
717  2022      7   30    22  32834.82
718  2022      7   30    23  30850.04
719  2022      7   30    24  28822.45

[720 rows x 5 columns]


In [ ]:
#For each day, keep only the CA ISO-TAC AREA_TYPE
df = df[df['AREA_TYPE'] == 'CA ISO-TAC']
# Group by OPR_DT, OPR_HR, and RENEWABLE_TYP
grouped_df.drop(columns=['TRADING_HUB'], inplace=True)
# Extract Year, Month, and Day from OPR_DT
grouped_df['Year'] = grouped_df['OPR_DT'].str[:4].astype(int)
grouped_df['Month'] = grouped_df['OPR_DT'].str[5:7].astype(int)
grouped_df['Day'] = grouped_df['OPR_DT'].str[8:10].astype(int)

# Reorder columns to place Year, Month, and Day at the beginning
columns_order = ['Year', 'Month', 'Day'] + [col for col in grouped_df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
grouped_df = grouped_df[columns_order]
grouped_df = grouped_df.rename(columns={'OPR_HR':'Hour'})

# Display the updated DataFrame
print(grouped_df)

In [44]:


# Define the base folder containing all monthly subfolders for load forecasts
base_dir = r"CAISO_DAM_Load_Forecasts"

# Function to process a single CSV file
def process_load_csv(file_path):
    # Read the file
    forecast_data = pd.read_csv(file_path)
    
    # Drop unnecessary columns
    columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                       'LOAD_TYPE', 'MARKET_RUN_ID', 'LABEL', 'XML_DATA_ITEM', 'POS', 'EXECUTION_TYPE', 'GROUP']
    df = forecast_data.drop(columns=columns_to_drop)
    
    # Filter for rows where TAC_AREA_NAME is 'CA ISO-TAC'
    df = df[df['TAC_AREA_NAME'] == 'CA ISO-TAC']
    df = df.reset_index(drop=True)
    df.drop(columns=['TAC_AREA_NAME'], inplace=True)
    
    # Sort by OPR_DT and OPR_HR
    df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)
    
    # Extract Year, Month, and Day from OPR_DT
    df['Year'] = df['OPR_DT'].str[:4].astype(int)
    df['Month'] = df['OPR_DT'].str[5:7].astype(int)
    df['Day'] = df['OPR_DT'].str[8:10].astype(int)
    
    # Reorder columns to place Year, Month, and Day at the beginning
    columns_order = ['Year', 'Month', 'Day'] + [col for col in df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
    df = df[columns_order]
    
    # Rename OPR_HR to Hour
    df = df.rename(columns={'OPR_HR': 'Hour'})
    
    return df

# Iterate through all subfolders and process each CSV file
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):  # Check if it's a folder
        # Extract the date (e.g., "2022_07") from the folder name
        folder_date = folder.split('_')[-2:]  # Extract the last two parts (e.g., "2022" and "07")
        folder_date_str = '_'.join(folder_date)  # Join them back as "2022_07"
        
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(folder_path, file)
                print(f"Processing file: {file_path}")
                
                # Process the CSV file
                processed_df = process_load_csv(file_path)
                
                # Save the processed DataFrame to a new CSV file
                output_file = os.path.join(folder_path, f"CAISO_DAM_Load_Forecast_{folder_date_str}_processed.csv")
                processed_df.to_csv(output_file, index=False)
                print(f"✅ Processed and saved: {output_file}")

Processing file: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_07\20220701_20220731_SLD_FCST_DAM_20251012_14_55_39_v1.csv
✅ Processed and saved: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_07\CAISO_DAM_Load_Forecast_2022_07_processed.csv
Processing file: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_08\20220801_20220831_SLD_FCST_DAM_20251012_14_55_45_v1.csv
✅ Processed and saved: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_08\CAISO_DAM_Load_Forecast_2022_08_processed.csv
Processing file: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_09\20220901_20220930_SLD_FCST_DAM_20251012_14_55_51_v1.csv
✅ Processed and saved: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_09\CAISO_DAM_Load_Forecast_2022_09_processed.csv
Processing file: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_10\20221001_20221031_SLD_FCST_DAM_20251012_14_56_01_v1.csv
✅ Processed and saved: CAISO_DAM_Load_Forecasts\CAISO_DAM_Load_Forecast_2022_10\CAISO_DAM_Load_Forecas

In [45]:
import zipfile
# Define the folder containing the .zip files
input_dir = r"CAISO_LMP_Forecasts"


# Extract all .zip files into separate subfolders
def extract_zip_files(input_dir):
    for file in os.listdir(input_dir):
        if file.endswith(".zip"):
            zip_path = os.path.join(input_dir, file)
            extract_folder = os.path.join(input_dir, file.replace(".zip", ""))  # Create a subfolder for each .zip file
            os.makedirs(extract_folder, exist_ok=True)  # Ensure the subfolder exists
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)  # Extract into the subfolder
            print(f"✅ Extracted: {file} to {extract_folder}")

# Call the function to extract files
extract_zip_files(input_dir)

✅ Extracted: CAISO_LMP_Forecast_2022_07.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_07
✅ Extracted: CAISO_LMP_Forecast_2022_08.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_08
✅ Extracted: CAISO_LMP_Forecast_2022_09.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_09
✅ Extracted: CAISO_LMP_Forecast_2022_10.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_10
✅ Extracted: CAISO_LMP_Forecast_2022_11.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_11
✅ Extracted: CAISO_LMP_Forecast_2022_12.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_12
✅ Extracted: CAISO_LMP_Forecast_2023_01.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2023_01
✅ Extracted: CAISO_LMP_Forecast_2023_02.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2023_02
✅ Extracted: CAISO_LMP_Forecast_2023_03.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2023_03
✅ Extracted: CAISO_LMP_Forecast_2023_04.zip to CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2023_04
✅ Extracted: CAISO_LMP_Forecast_2023_05.zip to CAISO_LMP_For

In [59]:
# Define the path to the file
file_path = r"CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_07\20220701_20220731_PRC_LMP_DAM_20251012_16_54_24_v12.csv"
# Read the file using pandas
forecast_data = pd.read_csv(file_path)
columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                   'NODE_ID_XML','NODE_ID', 'MARKET_RUN_ID', 'XML_DATA_ITEM', 'PNODE_RESMRID', 'GRP_TYPE', 'POS','GROUP']
# Drop the columns
df = forecast_data.drop(columns=columns_to_drop)
#Only keep for each hour, the 'LMP_TYPE=LMP' rows
df = df[df['LMP_TYPE'] == 'LMP']
df.drop(columns=['LMP_TYPE'], inplace=True)
df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)
#Group by hour and take the mean LMP across all nodes
df = df.groupby(['OPR_DT', 'OPR_HR'], as_index=False).sum()
#divide the LMP column by 3
df['LMP'] = df['MW'] / 3
df.drop(columns=['NODE','MW'], inplace=True)
#Extract Year, Month, Day from OPR_DT
df['Year'] = df['OPR_DT'].str[:4].astype(int)
df['Month'] = df['OPR_DT'].str[5:7].astype(int)
df['Day'] = df['OPR_DT'].str[8:10].astype(int)
#Reorder columns to place Year, Month, Day at the beginning
columns_order = ['Year', 'Month', 'Day'] + [col for col in df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
df = df[columns_order]
df = df.rename(columns={'OPR_HR':'Hour'})
# Display the updated DataFrame
print(df.head())

   Year  Month  Day  Hour        LMP
0  2022      7    1     1  66.762893
1  2022      7    1     2  62.648637
2  2022      7    1     3  61.347333
3  2022      7    1     4  59.363743
4  2022      7    1     5  59.856077


In [60]:
import os
import pandas as pd

# Define the base folder containing all monthly subfolders
base_dir = r"CAISO_LMP_Forecasts"

# Function to process a single CSV file
def process_lmp_csv(file_path):
    # Read the file
    forecast_data = pd.read_csv(file_path)
    
    # Drop unnecessary columns
    columns_to_drop = ['OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 
                       'NODE_ID_XML', 'NODE_ID', 'MARKET_RUN_ID', 'XML_DATA_ITEM', 
                       'PNODE_RESMRID', 'GRP_TYPE', 'POS', 'GROUP']
    df = forecast_data.drop(columns=columns_to_drop)
    
    # Only keep rows where LMP_TYPE is 'LMP'
    df = df[df['LMP_TYPE'] == 'LMP']
    df.drop(columns=['LMP_TYPE'], inplace=True)
    
    # Sort by OPR_DT and OPR_HR
    df = df.sort_values(by=['OPR_DT', 'OPR_HR'], ascending=[True, True]).reset_index(drop=True)
    
    # Group by OPR_DT and OPR_HR, and take the mean LMP across all nodes
    df = df.groupby(['OPR_DT', 'OPR_HR'], as_index=False).sum()
    
    # Divide the LMP column by 3
    df['LMP'] = df['MW'] / 3
    df.drop(columns=['NODE', 'MW'], inplace=True)
    
    # Extract Year, Month, and Day from OPR_DT
    df['Year'] = df['OPR_DT'].str[:4].astype(int)
    df['Month'] = df['OPR_DT'].str[5:7].astype(int)
    df['Day'] = df['OPR_DT'].str[8:10].astype(int)
    
    # Reorder columns to place Year, Month, and Day at the beginning
    columns_order = ['Year', 'Month', 'Day'] + [col for col in df.columns if col not in ['Year', 'Month', 'Day', 'OPR_DT']]
    df = df[columns_order]
    
    # Rename OPR_HR to Hour
    df = df.rename(columns={'OPR_HR': 'Hour'})
    
    return df

# Iterate through all subfolders and process each CSV file
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):  # Check if it's a folder
        # Extract the date (e.g., "2022_07") from the folder name
        folder_date = folder.split('_')[-2:]  # Extract the last two parts (e.g., "2022" and "07")
        folder_date_str = '_'.join(folder_date)  # Join them back as "2022_07"
        
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(folder_path, file)
                print(f"Processing file: {file_path}")
                
                # Process the CSV file
                processed_df = process_lmp_csv(file_path)
                
                # Save the processed DataFrame to a new CSV file
                output_file = os.path.join(folder_path, f"CAISO_LMP_Forecast_{folder_date_str}_processed.csv")
                processed_df.to_csv(output_file, index=False)
                print(f"✅ Processed and saved: {output_file}")

Processing file: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_07\20220701_20220731_PRC_LMP_DAM_20251012_16_54_24_v12.csv
✅ Processed and saved: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_07\CAISO_LMP_Forecast_2022_07_processed.csv
Processing file: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_08\20220801_20220831_PRC_LMP_DAM_20251012_16_54_32_v12.csv
✅ Processed and saved: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_08\CAISO_LMP_Forecast_2022_08_processed.csv
Processing file: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_09\20220901_20220930_PRC_LMP_DAM_20251012_16_54_41_v12.csv
✅ Processed and saved: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_09\CAISO_LMP_Forecast_2022_09_processed.csv
Processing file: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_10\20221001_20221031_PRC_LMP_DAM_20251012_16_54_50_v12.csv
✅ Processed and saved: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_10\CAISO_LMP_Forecast_2022_10_processed.csv
Processing file: CAISO_LMP_Forecasts\CAISO_LMP_Forecast_2022_11\20221101_202

In [7]:
FR_lmp=pd.read_csv(r"FR_LMP_Forecasts\dayahead_prices_FR_2022_2025.csv")
FR_lmp.rename(columns={'Unnamed: 0':'Time', 'EUR_per_MWh':'EUR/MWh'}, inplace=True)
FR_lmp['Year'] = FR_lmp['Time'].str[:4].astype(int)
FR_lmp['Month'] = FR_lmp['Time'].str[5:7].astype(int)
FR_lmp['Day'] = FR_lmp['Time'].str[8:10].astype(int)
FR_lmp['Hour'] = FR_lmp['Time'].str[11:13].astype(int)
FR_lmp['Timezone'] = FR_lmp['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
FR_lmp = FR_lmp[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'EUR/MWh']]
FR_lmp.to_csv(r"FR_LMP_Forecasts\FR_lmp_processed.csv", index=False)
print("France LMP save to FR_lmp_processed.csv")

France LMP save to FR_lmp_processed.csv


In [8]:
FR_wind=pd.read_csv(r"FR_REN_DAM_Forecasts\dayahead_wind_forecast_FR_2022_2025.csv")
FR_sun=pd.read_csv(r"FR_REN_DAM_Forecasts\dayahead_solar_forecast_FR_2022_2025.csv")
FR_wind.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
FR_sun.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
FR_wind['Year'] = FR_wind['Time'].str[:4].astype(int)
FR_wind['Month'] = FR_wind['Time'].str[5:7].astype(int)
FR_wind['Day'] = FR_wind['Time'].str[8:10].astype(int)
FR_wind['Hour'] = FR_wind['Time'].str[11:13].astype(int)
FR_wind['Timezone'] = FR_wind['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
FR_wind = FR_wind[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'Wind Onshore']]

FR_sun['Year'] = FR_sun['Time'].str[:4].astype(int)
FR_sun['Month'] = FR_sun['Time'].str[5:7].astype(int)
FR_sun['Day'] = FR_sun['Time'].str[8:10].astype(int)
FR_sun['Hour'] = FR_sun['Time'].str[11:13].astype(int)
FR_sun['Timezone'] = FR_sun['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
FR_sun = FR_sun[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'Solar']]

#Merge the two dataframes on Time, Year, Month, Day, Hour, Timezone
FR_ren = pd.merge(FR_wind, FR_sun, on=['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone'])
FR_ren.to_csv(r"FR_REN_DAM_Forecasts\FR_ren_processed.csv", index=False)
print("France REN save to FR_ren_processed.csv")

France REN save to FR_ren_processed.csv


In [9]:
FR_load=pd.read_csv(r"FR_DAM_Load_Forecasts\dayahead_load_forecast_FR_2022_2025.csv")
FR_load.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
FR_load['Year'] = FR_load['Time'].str[:4].astype(int)
FR_load['Month'] = FR_load['Time'].str[5:7].astype(int)
FR_load['Day'] = FR_load['Time'].str[8:10].astype(int)
FR_load['Hour'] = FR_load['Time'].str[11:13].astype(int)
FR_load['Timezone'] = FR_load['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
FR_load = FR_load[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'MW']]
FR_load.to_csv(r"FR_DAM_Load_Forecasts\FR_load_processed.csv", index=False)
print("France Load save to FR_load_processed.csv")

France Load save to FR_load_processed.csv


In [5]:
DE_lmp=pd.read_csv(r"DE_LMP_Forecasts\dayahead_prices_forecast_DE_2022_2025.csv")
DE_lmp.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
DE_lmp['Year'] = DE_lmp['Time'].str[:4].astype(int)
DE_lmp['Month'] = DE_lmp['Time'].str[5:7].astype(int)
DE_lmp['Day'] = DE_lmp['Time'].str[8:10].astype(int)
DE_lmp['Hour'] = DE_lmp['Time'].str[11:13].astype(int)
DE_lmp['Timezone'] = DE_lmp['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
DE_lmp = DE_lmp[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'EUR/MWh']]
DE_lmp.to_csv(r"DE_LMP_Forecasts\DE_lmp_processed.csv", index=False)
print("Germany LMP save to DE_lmp_processed.csv")

Germany LMP save to DE_lmp_processed.csv


In [10]:
DE_load=pd.read_csv(r"DE_DAM_Load_Forecasts\dayahead_load_forecast_DE_2022_2025.csv")
DE_load.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
DE_load['Year'] = DE_load['Time'].str[:4].astype(int)
DE_load['Month'] = DE_load['Time'].str[5:7].astype(int)
DE_load['Day'] = DE_load['Time'].str[8:10].astype(int)
DE_load['Hour'] = DE_load['Time'].str[11:13].astype(int)
DE_load['Timezone'] = DE_load['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
DE_load = DE_load[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'MW']]
DE_load.to_csv(r"DE_DAM_Load_Forecasts\DE_load_processed.csv", index=False)
print("Germany Load save to DE_load_processed.csv")

Germany Load save to DE_load_processed.csv


In [11]:
DE_wind=pd.read_csv(r"DE_REN_DAM_Forecasts\dayahead_wind_forecast_DE_2022_2025.csv")
DE_sun=pd.read_csv(r"DE_REN_DAM_Forecasts\dayahead_solar_forecast_DE_2022_2025.csv")
DE_wind.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
DE_sun.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
DE_wind['Year'] = DE_wind['Time'].str[:4].astype(int)
DE_wind['Month'] = DE_wind['Time'].str[5:7].astype(int)
DE_wind['Day'] = DE_wind['Time'].str[8:10].astype(int)
DE_wind['Hour'] = DE_wind['Time'].str[11:13].astype(int)
DE_wind['Timezone'] = DE_wind['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
DE_wind = DE_wind[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'Wind Onshore']]

DE_sun['Year'] = DE_sun['Time'].str[:4].astype(int)
DE_sun['Month'] = DE_sun['Time'].str[5:7].astype(int)
DE_sun['Day'] = DE_sun['Time'].str[8:10].astype(int)
DE_sun['Hour'] = DE_sun['Time'].str[11:13].astype(int)
DE_sun['Timezone'] = DE_sun['Time'].str.extract(r'([+-]\d{2}:\d{2})$')
DE_sun = DE_sun[['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone', 'Solar']]

#Merge the two dataframes on Time, Year, Month, Day, Hour, Timezone
DE_ren = pd.merge(DE_wind, DE_sun, on=['Time', 'Year', 'Month', 'Day', 'Hour', 'Timezone'])
DE_ren.to_csv(r"DE_REN_DAM_Forecasts\DE_ren_processed.csv", index=False)
print("Germany REN save to DE_ren_processed.csv")

Germany REN save to DE_ren_processed.csv
